# 第10回 その3: ニューラルネットワークによる非線形識別
いよいよニューラルネットワークを実装します。  
ここでは中間層が一つ，中間層のノード数が2のニューラルネットワークを実装します。    

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/10"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/10/
# 現在のフォルダの中身を表示
%ls

`2class_data_nl.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data_nl.csv` を読み込みます。  
このデータはx1とx2の二次元データで，クラス0とクラス1の2クラスのどちらかに属しています。  

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data_nl.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

標準化を行い，データをプロットします。

In [ ]:
# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

# 以下は color_list = ['b', 'r'] とほぼ同じ。
# 光の三原色(赤,緑,青)の値それぞれを0～1の範囲で定義している。
color_list = [(0, 0, 1), (1, 0, 0)]

# データをプロット
plt.figure(figsize=(7,7))
for k in range(num_classes):
  # n番目のクラスに属している(Y==c)データをプロット
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

このデータ`2class_data_nl.csv`は**直線で2クラスに識別することはできません**。  
つまり，<font color=red>**線形分離不可能なデータ**</font>ということになります。  
このようなデータに対しては，ロジスティック回帰や`09_01_softmax_3class.ipynb`で実装した方法のような**線形モデル**では100%の精度で識別することができません。  
そこでここでは**非線形モデル** (non-linear model) であるニューラルネットワークを使用して識別することにします。  


## ステップ2: ニューラルネットワークの実装  
ここでは，中間層の数を1，中間層のノードの数を2としたニューラルネットワークを実装します。  
このモデルの式をは以下のようになります。  

$\left[\begin{matrix}g_{1}\\g_{2} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}^{\rm mid}x_1 + w_{12}^{\rm mid}x_2 + b_{1}^{\rm mid}\\ w_{21}^{\rm mid}x_1 + w_{22}^{\rm mid}x_2 + b_{2}^{\rm mid} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}^{\rm mid} & w_{12}^{\rm mid} \\ w_{21}^{\rm mid} & w_{22}^{\rm mid} \end{matrix} \right]\left[\begin{matrix}x_1\\ x_2\end{matrix} \right] + \left[\begin{matrix}b_{1}^{\rm mid}\\b_{2}^{\rm mid} \end{matrix} \right] = {\bf W}^{\rm mid}{\bf x} + {\bf b}^{\rm mid}$  
$\left[\begin{matrix}h_{1}\\h_{2} \end{matrix} \right] = 
\left[\begin{matrix}{\rm sigmoid}(g_1) \\ {\rm sigmoid}(g_2)\end{matrix} \right]$  
$\left[\begin{matrix}z_{1}\\z_{2} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}h_1 + w_{12}h_2 + b_{1}\\ w_{21}h_1 + w_{22}h_2 + b_{2} \end{matrix} \right] = 
\left[\begin{matrix} w_{11} & w_{12} \\ w_{21} & w_{22} \end{matrix} \right]\left[\begin{matrix}h_1\\h_2 \end{matrix} \right] + \left[\begin{matrix}b_{1}\\ b_{2} \end{matrix} \right] = {\bf W}{\bf h} + {\bf b}$  
$\left[\begin{matrix}\hat{y}_{1}\\\hat{y}_{2} \end{matrix} \right] = 
\left[\begin{matrix}{\rm softmax}(z_1) \\ {\rm softmax}(z_2)\end{matrix} \right]$

${\bf W}^{\rm mod}$は中間層における重み行列で，サイズは ノード数 $\times$ 次元数 = $2 \times 2$です。  
${\bf b}^{\rm mod}$ は中間層における切片（バイアス）ベクトルで，サイズは ノード数$=2$ です。    
${\bf W}$は出力層における重み行列で，サイズは クラス数 $\times$ 中間層のノード数 = $2 \times 2$です。  
${\bf b}$ は出力層における切片（バイアス）ベクトルで，サイズは クラス数$=2$ です。  
全部サイズが2ですが，それぞれ次元数，ノード数，クラス数と意味が違うので注意してください。（レポート課題で重要になるかもしれません。）

1サンプルデータに対してニューラルネットワークを通し，$\bf \hat{y}$ を計算する関数を以下に定義します。  
ちなみに，以下の neural_network 関数は中間層のノード数が2の場合でしか使えない（汚い）実装になっています。  

In [ ]:
def sigmoid(z):
  '''
      シグモイド関数
  '''
  return (1+np.exp(-1*z))**(-1)

def softmax(z):
  '''
      ソフトマックス関数
      z: 要素数=クラス数のベクトル
      y_hat: クラス毎の確率。要素数=クラス数のベクトル
  '''
  return (np.exp(z) / np.sum(np.exp(z)))

def neural_network(x, w_mid_1, b_mid_1, w_mid_2, b_mid_2, W, b):
  '''
      1サンプルデータに対してニューラルネットワークによるクラス分類を行う
      x: 1サンプルデータ。サイズ=Dのベクトル
      w_mid_1: 中間層のノード1に関する重み行列。サイズ=Dのベクトル
      b_mid_1: 中間層のノード1に関するバイアス。スカラ
      w_mid_2: 中間層のノード2に関する重み行列。サイズ=Dのベクトル
      b_mid_2: 中間層のノード2に関するバイアス。スカラ
      W: 出力層の重み行列。サイズ=(KxJ)の行列
      b: 出力層のバイアス。サイズ=Kの行列
      ただし，D:次元数，J:中間層のノード数，K:クラス数である。
      y_hat: 出力層の出力。サイズ=Kのベクトル
      h1: 中間層のノード1の出力。スカラ
      h2: 中間層のノード2の出力。スカラ
  '''
  # 中間層を計算
  # ノード1を計算
  g1 = np.dot(w_mid_1, x) + b_mid_1
  h1 = sigmoid(g1)
  # ノード2を計算
  g2 = np.dot(w_mid_2, x) + b_mid_2
  h2 = sigmoid(g2)

  # 出力層を計算
  z = np.dot(W, np.array([h1, h2])) + b
  y_hat = softmax(z)

  return y_hat, h1, h2


neural_network関数の`w_mid_1`は$\left[\begin{matrix} w_{11}^{\rm mid} & w_{12}^{\rm mid} \end{matrix}\right]$，`w_mid_2`は$\left[\begin{matrix} w_{21}^{\rm mid} & w_{22}^{\rm mid} \end{matrix}\right]$のことです。  
`b_mid_1`，`b_mid_2`はそれぞれ$b_{1}^{\rm mid}, b_{2}^{\rm mid}$のことです。  
`W`，`b`はそれぞれ$\bf W$，$\bf b$のことです。

各パラメータの初期値を決めます。  
ロジスティック回帰のときと同様，${\bf b}^{\rm mod}$ および ${\bf b}$ の初期値はゼロ，${\bf W}^{\rm mod}$ および ${\bf W}$ の初期値は正規分布に従う乱数で決定します。   

In [ ]:
np.random.seed(0)

# 中間層の初期値
# ノード1
initial_w_mid_1 =  np.random.randn(num_dimensions)
initial_b_mid_1 = 0
# ノード2
initial_w_mid_2 =  np.random.randn(num_dimensions)
initial_b_mid_2 = 0

# 出力層の初期値
initial_W =  np.random.randn(num_classes, 2) # ここの2はノード数の2に相当
initial_b = np.zeros(num_classes)

print('initial_w_mid_1 = ')
print(initial_w_mid_1)
print('initial_w_mid_2 = ')
print(initial_w_mid_2)
print('initial_W = ')
print(initial_W)

クロスエントロピー損失を計算する関数および勾配を計算する関数を定義します。  
勾配の計算式は以下のように定義されます。  
$\frac{\partial L_{ce}}{\partial {\bf W}_{k}} = (\hat{y}_k - y_k){\bf h}$  
$\frac{\partial L_{ce}}{\partial {\bf b}_{k}} = \hat{y}_k - y_k$  
$\frac{\partial L_{ce}}{\partial {\bf W}_{j}^{\rm mod}} = \left [ \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj} \right ] (1-h_j)h_j{\bf x}$   
$\frac{\partial L_{ce}}{\partial {\bf b}_{j}^{\rm mod}} = \left [ \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj} \right ] (1-h_j)h_j$ 

$k$は出力層のノード番号（つまりクラス番号），$j$は中間層のノード番号です。   
$y_k$は正解ラベルで，$k$が正解クラスの場合は確率1，不正解クラスの場合は0です。  

なお，関数 calc_gradientも中間層のノード数が2の時にしか使えない（汚い）実装になっています。

In [ ]:
def cross_entropy(y_hat, y):
  '''
      クロスエントロピー損失を計算する。
      y_hat: マルチクラス回帰によって推定された y
      y: 正解のクラス
  '''
  # 正解クラスの確率を1，それ以外のクラスの確率を0とするベクトル（one-hotベクトル）を作成する。
  y_onehot = np.zeros(num_classes)
  y_onehot[y] = 1.0

  ce = -1.0 * np.sum(y_onehot*np.log(y_hat))

  return ce

def calc_gradient(y_hat, y, h1, h2, W, x):
  '''
      勾配を計算する
      y_hat: 出力層の出力: サイズ=Kのベクトル
      y: 正解のクラス番号: スカラ
      h1: 中間層のノード1の出力。スカラ
      h2: 中間層のノード2の出力。スカラ
      W: 出力層の重み行列。サイズ=(KxJ)の行列
      x: データ: サイズ=Dのベクトル
      ただし，D:次元数，J:中間層のノード数，K:クラス数である。
      grad_w_mid_1: w_mid_1の勾配：サイズ=Dのベクトル
      grad_b_mid_1: b_mid_1の勾配：スカラ
      grad_w_mid_2: w_mid_2の勾配：サイズ=Dのベクトル
      grad_b_mid_2: b_mid_2の勾配：スカラ
      grad_W: Wの勾配: サイズ=(KxJ)の行列
      grad_b: bの勾配: サイズ=Kのベクトル
  '''
  # 次元数D, 中間層のノード数J, クラス数Kを得る。
  D = np.size(x)
  J = 2
  K = np.size(y_hat)

  # 正解クラスの確率を1，それ以外のクラスの確率を0とするベクトル（one-hotベクトル）を作成する。
  y_onehot = np.zeros(K)
  y_onehot[y] = 1.0

  # 出力層の勾配を計算
  grad_W = np.zeros([K, J])
  grad_b = np.zeros(K)
  for k in range(K):
    grad_W[k,:] = (y_hat[k] - y_onehot[k]) * np.array([h1, h2])
    grad_b[k] = y_hat[k] - y_onehot[k]

  # 中間層の勾配を計算
  # ノード1(W[:,0],h1を使っている点に注意)
  grad_w_mid_1 = np.sum((y_hat - y_onehot)*W[:,0])*(1 - h1)*h1 * x
  grad_b_mid_1 = np.sum((y_hat - y_onehot)*W[:,0])*(1 - h1)*h1
  # ノード2(W[:,1],h2を使っている点に注意)
  grad_w_mid_2 = np.sum((y_hat - y_onehot)*W[:,1])*(1 - h2)*h2 * x
  grad_b_mid_2 = np.sum((y_hat - y_onehot)*W[:,1])*(1 - h2)*h2
  
  return grad_w_mid_1, grad_b_mid_1, grad_w_mid_2, grad_b_mid_2, grad_W, grad_b

ではロジスティック回帰のときと同様に，全てのデータを用いて繰り返し更新しましょう。   
ここでは全データサンプルに対して更新を行うひとまとまりの処理を60周（エポック）行っています。

In [ ]:
from matplotlib import animation, rc
from IPython.display import HTML

np.random.seed(0)

# 学習率
lr = 0.2

# パラメータの初期化
w_mid_1 = initial_w_mid_1.copy()
b_mid_1 = initial_b_mid_1
w_mid_2 = initial_w_mid_2.copy()
b_mid_2 = initial_b_mid_2
W = initial_W.copy()
b = initial_b.copy()

# 描画
color_list = [(0, 0, 1), (1, 0, 0)]
fig = plt.figure(figsize=(7,7))
# データの描画
for k in range(num_classes):
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
images = []

# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

for epoch in range(60):
  
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n,:]
    y = Y_tmp[n]
  
    # ニューラルネットワークに与える
    y_hat, h1, h2 = neural_network(x, w_mid_1, b_mid_1, w_mid_2, b_mid_2, W, b)
  
    # 勾配の計算
    grad_w_mid_1, grad_b_mid_1, grad_w_mid_2, grad_b_mid_2, grad_W, grad_b = calc_gradient(y_hat, y, h1, h2, W, x)

    # 2エポック毎にアニメーション描画
    if n == 0 and epoch % 2 == 0:
      img = []
      x1 = np.linspace(-3, 3)
      x2 = -1.0 * (b_mid_1 + w_mid_1[0]*x1) / w_mid_1[1]
      img += plt.plot(x1, x2, color='k')
      x2 = -1.0 * (b_mid_2 + w_mid_2[0]*x1) / w_mid_2[1]
      img += plt.plot(x1, x2, color='k')
      img.append(plt.text(-2.8, 2.7, 'epoch: '+str(epoch), size='x-large'))
      images.append(img)

    # 更新
    W -= lr * grad_W
    b -= lr * grad_b
    w_mid_1 -= lr * grad_w_mid_1
    b_mid_1 -= lr * grad_b_mid_1
    w_mid_2 -= lr * grad_w_mid_2
    b_mid_2 -= lr * grad_b_mid_2

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y)
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))

plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])

# アニメーション作成
anim = animation.ArtistAnimation(fig, images, interval=200)
# Google Colaboratoryの場合必要
rc('animation', html='jshtml')
plt.close()
display(anim)

上記のアニメ－ションでは，中間層の2個のノードそれぞれに関する直線をプロットしています。  

エポック毎の損失関数（クロスエントロピー）および分類正解率をプロットします。  

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

（エポック毎に計算していますが）学習完了後の分類正解率を計算します。  

In [ ]:
# 正解率
accuracy = 0

for n in range(num_samples):
  x = X[n,:]
  y = Y[n]
  # ニューラルネットワークに与える
  y_hat, h1, h2 = neural_network(x, w_mid_1, b_mid_1, w_mid_2, b_mid_2, W, b)

  if np.argmax(y_hat) == y:
    accuracy += 1

accuracy = 100.0 * accuracy / num_samples

print('Accuracy = %.3f' % (accuracy))

100%の正解率が得られました。

最後に，識別境界を可視化します。  
今回，中間層には2個のノードを設定しているので，中間層においては2本の直線が学習されていることになります。  
ここでは，識別境界に加えて，中間層で学習された2本の直線もプロットしてみます。  

In [ ]:
# x1，x2ごとの格子点の数
grid_size = 200

# 格子点：-3.1から3.1まで200個分，等間隔に区切った数列
grid = np.linspace(-3.1, 3.1, grid_size)

# クラス識別結果を色塗りする領域：200x200の図
# 最後の3は，色情報(RGB=赤・緑・青の3次元ごとの色の強さ)
image = np.zeros([grid_size, grid_size, 3])

color_list = [(0, 0, 1), (1, 0, 0)]

for i in range(grid_size):
  for j in range(grid_size):
    # 格子点のデータ
    x = np.array([grid[j], grid[i]])
    # 格子点データを識別する
    y_hat, h1, h2 = neural_network(x, w_mid_1, b_mid_1, w_mid_2, b_mid_2, W, b)
    label = np.argmax(y_hat)
    # 識別結果のクラスの色で，色を塗る
    # (グラフは原点が左下だが，画像は左上が原点なので，上下反転させるために-iとしている)
    image[-i, j, :] = color_list[label]

plt.figure(figsize=(7,7))
# 色塗り結果を表示
plt.imshow(image, alpha=0.4, extent=(-3.1, 3.1, -3.1, 3.1))
# データおよび中間層の識別補助線を表示
x1 = np.linspace(-3, 3)
x2 = -1.0 * (b_mid_1 + w_mid_1[0]*x1) / w_mid_1[1]
plt.plot(x1, x2, color='k')
x2 = -1.0 * (b_mid_2 + w_mid_2[0]*x1) / w_mid_2[1]
plt.plot(x1, x2, color='k')
for k in range(num_classes):
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

中間層の2本の直線によって，非線形な識別境界が作られていることが分かります。  
この例では，正の傾きを持つ直線より下で，かつ負の傾きを持つ直線よりも上にいるデータはクラス1（赤），それ以外はクラス0(青)といった感じの識別が行われています。  
このような，複数の直線による線形判別を組み合わせた判別は**区分的線形判別**と呼ばれることもあります。  
中間層が1層のニューラルネットワークは，近似的に区分的線形判別に等しいことが，この例から分かると思います。